**Index the DBpedia entities into elasticSearch based on ENTITY CENTRIC method**

In [1]:
import json
import pickle
from helper_function import preprocess, Indexer
from elasticsearch import Elasticsearch
from typing import Dict, List, Optional

In [2]:
#%run DBpedia_map_entity_abstract.ipynb

In [3]:
#conda install elasticsearch

In [4]:
INDEX_NAME = "dbpedia_entity_centric"
INDEX_SETTINGS = {
    "mappings": {
        "properties": {
            "entity": {
                "type": "text",
                "term_vector": "yes",
                "analyzer": "english",
            },
            "abstract": {
                "type": "text",
                "term_vector": "yes",
                "analyzer": "english",
            },

        }
    }
}


In [28]:
def index_dbpdiea_type_centric(index:str, index_settings:Dict,reset=False,
                                   filepath="../data/short_abstracts_en.ttl",
                                   es=Elasticsearch())->bool:
    """
    set reset to be True, if want to reset and re-index into the elasticSearch
    """
    es.info()
    if reset and es.indices.exists(index):
        es.indices.delete(index=index)
        print(f'"{index}" is reset.')
        
    if es.indices.exists(index):
        count = es.cat.count(index, params={"format": "json"})
        print(f'"{index}" already exists in ElasticSearch, and there are {int(count[0]["count"])} items.') 
        return True
    try:
        file_object = open(filepath,"r",encoding="utf8") 
        file = file_object.read()
#         with open(filepath, 'r', encoding='utf-8') as f: 
#             file=f
        print(f'"{filepath}" is loaded.') 
 
    except FileNotFoundError:
        print(f'"{filepath}" is not found.')
        return False
    
    
    #process file and index into elasticSearch
    es.indices.create(index=index, body=index_settings)
    i=0
    j=0
    with open(filepath, 'r', encoding='utf-8') as file: 
        for line in file:
            j+=1
            if j%100000==0:
                print("-----------j",j)
            if (line is None) or (line[0] == '#'):
                continue
            #line = line.lower().strip()[:-5].replace('/>', '>').split(' ')
            line=line.split(' ')
            if len(line) < 3:
                continue
            entity = line[0][1:-1].split('/')[-1]
            entity=preprocess(entity)
            line[-2]=line[-2][:-4]
            abstract = preprocess(' '.join(line[2:-1]).replace('\\', ''))
            doc={"entity":entity,"abstract":abstract}
            es.index(index=index, doc_type="_doc", id=str(i), body=doc)
            i+=1
            
    es.indices.refresh(index)
    count = es.cat.count(index, params={"format": "json"})          
    print(f'{j} entities are processed, \n{count[0]["count"]} entities are indexed into elastic search')
    return True


In [29]:
#test with small file
index="test_entity_centric"
index_settings=INDEX_SETTINGS
filepath="../data/xaa_small_abs"
index_dbpdiea_type_centric(index, index_settings,
                           reset=True,
                           filepath=filepath,
                           es=Elasticsearch())
es = Elasticsearch()
es.info()
es.get(index=index,id="15")

C:\Users\junec\AppData\Local\Temp/ipykernel_13540/3375190284.py:8: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if reset and es.indices.exists(index):


"test_entity_centric" is reset.
"../data/xaa_small_abs" is loaded.


C:\Users\junec\AppData\Local\Temp/ipykernel_13540/3375190284.py:12: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(index):
C:\Users\junec\AppData\Local\Temp/ipykernel_13540/3375190284.py:29: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=index, body=index_settings)
C:\Users\junec\AppData\Local\Temp/ipykernel_13540/3375190284.py:48: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index

6000 entities are processed, 
5999 entities are indexed into elastic search


{'_index': 'test_entity_centric',
 '_type': '_doc',
 '_id': '15',
 '_version': 1,
 '_seq_no': 15,
 '_primary_term': 1,
 'found': True,
 '_source': {'entity': 'Appellate procedure in the United States',
  'abstract': ' United States appellate procedure involves the rules and regulations for filing appeals in state courts and federal courts  The nature of an appeal can vary greatly depending on the type of case and the rules of the court in the jurisdiction where the case was prosecuted  There are many types of standard of review for appeals  such as de novo and abuse of discretion  However  most appeals begin when a party files a petition for review to a higher court for the purpose of overturning the lower court s decision '}}

In [5]:
filepath="../data/short_abstracts_en.ttl"
index_dbpdiea_type_centric(index=INDEX_NAME, index_settings=INDEX_SETTINGS,
                           reset=False,
                           filepath=filepath,
                           es=Elasticsearch())

C:\Users\junec\AppData\Local\Temp/ipykernel_16056/3275183715.py:27: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME, doc_type="_doc", id=str(i), body=doc)


-----------i 0
-----------j 100000
-----------i 100000
-----------j 200000
-----------i 200000
-----------j 300000
-----------i 300000
-----------j 400000
-----------i 400000
-----------j 500000
-----------i 500000
-----------j 600000
-----------i 600000
-----------j 700000
-----------i 700000
-----------j 800000
-----------i 800000
-----------j 900000
-----------i 900000
-----------j 1000000
-----------i 1000000
-----------j 1100000
-----------i 1100000
-----------j 1200000
-----------i 1200000
-----------j 1300000
-----------i 1300000
-----------j 1400000
-----------i 1400000
-----------j 1500000
-----------i 1500000
-----------j 1600000
-----------i 1600000
-----------j 1700000
-----------i 1700000
-----------j 1800000
-----------i 1800000
-----------j 1900000
-----------i 1900000
-----------j 2000000
-----------i 2000000
-----------j 2100000
-----------i 2100000
-----------j 2200000
-----------i 2200000
-----------j 2300000
-----------i 2300000
-----------j 2400000
-----------i 240

In [27]:
filepath="../data/short_abstracts_en.ttl"
index_dbpdiea_type_centric(index=INDEX_NAME, index_settings=INDEX_SETTINGS,
                           reset=False,
                           filepath=filepath,
                           es=Elasticsearch())
es = Elasticsearch()
es.info()
es.get(index=INDEX_NAME,id="15")

"dbpedia_entity_centric" already exists in ElasticSearch, and there are 4935279 items.


C:\Users\junec\AppData\Local\Temp/ipykernel_13540/2749173149.py:12: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(index):
C:\Users\junec\AppData\Local\Temp/ipykernel_13540/2749173149.py:13: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  count = es.cat.count(index, params={"format": "json"})


{'_index': 'dbpedia_entity_centric',
 '_type': '_doc',
 '_id': '15',
 '_version': 3,
 '_seq_no': 12013,
 '_primary_term': 1,
 'found': True,
 '_source': {'entity': 'Appellate procedure in the United States',
  'abstract': ' United States appellate procedure involves the rules and regulations for filing appeals in state courts and federal courts  The nature of an appeal can vary greatly depending on the type of case and the rules of the court in the jurisdiction where the case was prosecuted  There are many types of standard of review for appeals  such as de novo and abuse of discretion  However  most appeals begin when a party files a petition for review to a higher court for the purpose of overturning the lower court s decision '}}

In [7]:
term="appellate" 
field="abstract"
hits = (
        es.search(
            index=INDEX_NAME,
            query={"match": {field: term}},
            _source=False,
            size=1,
        )
        .get("hits", {})
        .get("hits", {})
    )
hits

[{'_index': 'dbpedia_entity_centric',
  '_type': '_doc',
  '_id': '714513',
  '_score': 13.31942}]